# Conditional Variational Auto Encoder (VAE) to project weather observation into the smaller-dimension latent space.

Data download: Met Office UK static 2km Numerical Weather Prediction (NWP) data.

In [ ]:
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from model.WeatherVAE import WeatherVAE
from common import common


In [2]:
common.init_notebook()

In [3]:
outdir = common.data_path("weather_vae_out_for_validation")
os.makedirs(outdir, exist_ok=True)

In [4]:
# The dimension of the latent space.
latent_dim = 8

# The units of the VAE Dense Layers.
units = [456, 44, 284, 328]

In [5]:
# Create WeatherVAE instance

weather_vae = WeatherVAE()
df = weather_vae.load_nwp_data()
df = weather_vae.feature_engineering(df)
df_std = weather_vae.scale_data(df)
used_columns = df_std.columns

In [6]:
used_columns.to_list()

['cloud_amount_of_high_cloud',
 'cloud_amount_of_low_cloud',
 'cloud_amount_of_medium_cloud',
 'cloud_amount_of_total_cloud',
 'fog_fraction_at_screen_level',
 'pressure_at_surface',
 'relative_humidity_at_screen_level',
 'temperature_at_screen_level',
 'visibility_at_screen_level',
 'rainfall_accumulation-PT01H',
 'snowfall_accumulation-PT01H',
 'wx',
 'wy']

## Reshape the DataFrame into the tensor

In [7]:
std_custom_set, label_set = weather_vae.prepare_dataset(df, df_std)

## Splitting the dataset

In [8]:
label_set = np.asarray(label_set)

In [9]:
def xy_data_to_df(X, y):
  df = pd.DataFrame(X.reshape(-1, len(used_columns)), columns=used_columns)
  df = weather_vae.unscale_data(df, used_columns)
  df['date_time'] = np.repeat(y, 1)
  return df

In [10]:
r2_rows = []

def append_r2score(x_train_df, decoded_imgs_train_df, x_test_df, decoded_imgs_test_df):
  r2_row = {}
  for j, weather_param in enumerate(used_columns):
    num_obs = len(x_train)
    y_train_true = x_train_df[weather_param].to_numpy().reshape(num_obs, -1)
    y_train_pred = decoded_imgs_train_df[weather_param].to_numpy().reshape(num_obs, -1)
    r2_train = r2_score(y_train_true, y_train_pred, multioutput='variance_weighted')  

    num_obs = len(x_test)
    y_test_true = x_test_df[weather_param].to_numpy().reshape(num_obs, -1)
    y_test_pred = decoded_imgs_test_df[weather_param].to_numpy().reshape(num_obs, -1)
    r2_test = r2_score(y_test_true, y_test_pred, multioutput='variance_weighted')

    r2_row[f"r2_train_{j:02d}"] = r2_train
    r2_row[f"r2_test_{j:02d}"] = r2_test

  r2_rows.append(r2_row)


## Repeat training and test of the WeatherVAE for 100 times

In [11]:
# Repeat 5-fold cross validation for 20 times (total 100)

for r in range(20):
  kf = KFold(n_splits=5, shuffle=True, random_state=r)
  for train_index, test_index in kf.split(std_custom_set):
    x_train, x_test = std_custom_set[train_index], std_custom_set[test_index]
    y_train, y_test = label_set[train_index], label_set[test_index]

    weather_vae.create_model(len(used_columns), latent_dim, units)
    vae2 = weather_vae.vae
    encoder = weather_vae.encoder
    decoder = weather_vae.decoder
    # Changed the optimizer due to this warning when it was run on M1 mac.
    # The 2nd argument of the compile function (loss) is the reconstruction loss.
    # Reconstruction loss calculates the error of decoded output from the input to the encoder.

    # It is compiled in the VAE_2D module.
    # optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    # vae2.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())

    time_train = weather_vae.create_time_input(y_train)
    time_test = weather_vae.create_time_input(y_test)

    vae2.fit([x_train, time_train], x_train, epochs=100, batch_size=32)
    decoded_imgs_test = vae2([x_test, time_test]).numpy()
    decoded_imgs_train = vae2([x_train, time_train]).numpy()

    x_test_df = xy_data_to_df(x_test, y_test)
    decoded_imgs_test_df = xy_data_to_df(decoded_imgs_test, y_test)

    x_train_df = xy_data_to_df(x_train, y_train)
    decoded_imgs_train_df = xy_data_to_df(decoded_imgs_train, y_train)

    append_r2score(x_train_df, decoded_imgs_train_df, x_test_df, decoded_imgs_test_df)

Epoch 1/100
219/219 [==============================] - 0s 988us/step - loss: 0.9313
Epoch 2/100
219/219 [==============================] - 0s 1ms/step - loss: 0.5810
Epoch 3/100
219/219 [==============================] - 0s 999us/step - loss: 0.4341
Epoch 4/100
219/219 [==============================] - 0s 959us/step - loss: 0.3582
Epoch 5/100
219/219 [==============================] - 0s 970us/step - loss: 0.3093
Epoch 6/100
219/219 [==============================] - 0s 978us/step - loss: 0.2770
Epoch 7/100
219/219 [==============================] - 0s 990us/step - loss: 0.2432
Epoch 8/100
219/219 [==============================] - 0s 991us/step - loss: 0.2129
Epoch 9/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1934
Epoch 10/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1821
Epoch 11/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1742
Epoch 12/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1793

Epoch 1/100
219/219 [==============================] - 0s 1ms/step - loss: 0.8687
Epoch 2/100
219/219 [==============================] - 0s 1ms/step - loss: 0.5742
Epoch 3/100
219/219 [==============================] - 0s 1ms/step - loss: 0.4461
Epoch 4/100
219/219 [==============================] - 0s 1ms/step - loss: 0.3680
Epoch 5/100
219/219 [==============================] - 0s 1ms/step - loss: 0.3225
Epoch 6/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2951
Epoch 7/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2677
Epoch 8/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2437
Epoch 9/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2162
Epoch 10/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2039
Epoch 11/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1833
Epoch 12/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1773
Epoch 13/100


Epoch 1/100
219/219 [==============================] - 1s 2ms/step - loss: 0.9085
Epoch 2/100
219/219 [==============================] - 0s 1ms/step - loss: 0.5686
Epoch 3/100
219/219 [==============================] - 0s 1ms/step - loss: 0.4452
Epoch 4/100
219/219 [==============================] - 0s 1ms/step - loss: 0.3375
Epoch 5/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2808
Epoch 6/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2501
Epoch 7/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2290
Epoch 8/100
219/219 [==============================] - 0s 2ms/step - loss: 0.2149
Epoch 9/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1978
Epoch 10/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1881
Epoch 11/100
219/219 [==============================] - 0s 2ms/step - loss: 0.1787
Epoch 12/100
219/219 [==============================] - 0s 2ms/step - loss: 0.1845
Epoch 13/100


Epoch 1/100
219/219 [==============================] - 1s 1ms/step - loss: 0.8959
Epoch 2/100
219/219 [==============================] - 0s 1ms/step - loss: 0.5674
Epoch 3/100
219/219 [==============================] - 0s 1ms/step - loss: 0.4837
Epoch 4/100
219/219 [==============================] - 0s 1ms/step - loss: 0.3965
Epoch 5/100
219/219 [==============================] - 0s 1ms/step - loss: 0.3440
Epoch 6/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2802
Epoch 7/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2558
Epoch 8/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2167
Epoch 9/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1988
Epoch 10/100
219/219 [==============================] - 0s 2ms/step - loss: 0.1922
Epoch 11/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1884
Epoch 12/100
219/219 [==============================] - 0s 2ms/step - loss: 0.1785
Epoch 13/100


Epoch 1/100
219/219 [==============================] - 0s 1ms/step - loss: 0.8771
Epoch 2/100
219/219 [==============================] - 0s 1ms/step - loss: 0.5293
Epoch 3/100
219/219 [==============================] - 0s 1ms/step - loss: 0.4442
Epoch 4/100
219/219 [==============================] - 0s 1ms/step - loss: 0.3625
Epoch 5/100
219/219 [==============================] - 1s 3ms/step - loss: 0.3056
Epoch 6/100
219/219 [==============================] - 0s 2ms/step - loss: 0.2784
Epoch 7/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2434
Epoch 8/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2174
Epoch 9/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1997
Epoch 10/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1864
Epoch 11/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1766
Epoch 12/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1722
Epoch 13/100


...

Epoch 1/100
219/219 [==============================] - 1s 1ms/step - loss: 0.8466
Epoch 2/100
219/219 [==============================] - 0s 1ms/step - loss: 0.5631
Epoch 3/100
219/219 [==============================] - 0s 1ms/step - loss: 0.4202
Epoch 4/100
219/219 [==============================] - 0s 1ms/step - loss: 0.3289
Epoch 5/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2821
Epoch 6/100
219/219 [==============================] - 0s 2ms/step - loss: 0.2443
Epoch 7/100
219/219 [==============================] - 0s 2ms/step - loss: 0.2251
Epoch 8/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2123
Epoch 9/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2142
Epoch 10/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1959
Epoch 11/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1851
Epoch 12/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1764
Epoch 13/100


Epoch 1/100
219/219 [==============================] - 1s 2ms/step - loss: 0.9029
Epoch 2/100
219/219 [==============================] - 0s 1ms/step - loss: 0.5766
Epoch 3/100
219/219 [==============================] - 0s 1ms/step - loss: 0.4149
Epoch 4/100
219/219 [==============================] - 0s 1ms/step - loss: 0.3679
Epoch 5/100
219/219 [==============================] - 0s 2ms/step - loss: 0.3262
Epoch 6/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2906
Epoch 7/100
219/219 [==============================] - 0s 2ms/step - loss: 0.2517
Epoch 8/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2258
Epoch 9/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2129
Epoch 10/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2136
Epoch 11/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1967
Epoch 12/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1901
Epoch 13/100


Epoch 1/100
219/219 [==============================] - 1s 1ms/step - loss: 0.9688
Epoch 2/100
219/219 [==============================] - 0s 1ms/step - loss: 0.5861
Epoch 3/100
219/219 [==============================] - 0s 1ms/step - loss: 0.4159
Epoch 4/100
219/219 [==============================] - 0s 1ms/step - loss: 0.3371
Epoch 5/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2648
Epoch 6/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2285
Epoch 7/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2171
Epoch 8/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2171
Epoch 9/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2004
Epoch 10/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1872
Epoch 11/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1742
Epoch 12/100
219/219 [==============================] - 0s 2ms/step - loss: 0.1701
Epoch 13/100


Epoch 1/100
219/219 [==============================] - 1s 2ms/step - loss: 0.9322
Epoch 2/100
219/219 [==============================] - 0s 1ms/step - loss: 0.5819
Epoch 3/100
219/219 [==============================] - 0s 1ms/step - loss: 0.4241
Epoch 4/100
219/219 [==============================] - 0s 1ms/step - loss: 0.3216
Epoch 5/100
219/219 [==============================] - 0s 2ms/step - loss: 0.2628
Epoch 6/100
219/219 [==============================] - 0s 2ms/step - loss: 0.2267
Epoch 7/100
219/219 [==============================] - 0s 2ms/step - loss: 0.2099
Epoch 8/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1973
Epoch 9/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1885
Epoch 10/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1931
Epoch 11/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1835
Epoch 12/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1734
Epoch 13/100


Epoch 1/100
219/219 [==============================] - 1s 1ms/step - loss: 0.8668
Epoch 2/100
219/219 [==============================] - 0s 2ms/step - loss: 0.5298
Epoch 3/100
219/219 [==============================] - 1s 3ms/step - loss: 0.4498
Epoch 4/100
219/219 [==============================] - 0s 1ms/step - loss: 0.3560
Epoch 5/100
219/219 [==============================] - 0s 1ms/step - loss: 0.3039
Epoch 6/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2811
Epoch 7/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2434
Epoch 8/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2217
Epoch 9/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2079
Epoch 10/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1972
Epoch 11/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1816
Epoch 12/100
219/219 [==============================] - 0s 1ms/step - loss: 0.1742
Epoch 13/100


## Output R2 Score

In [12]:

r2_df = pd.DataFrame(r2_rows)
r2_df.to_csv(f"{outdir}/weather_vae_r2_scores.csv")